# Assess Task Compute Time and Data Sizes
To get a better idea how they are each different and provide quantitative descriptions in the paper

In [1]:
from pathlib import Path
import pandas as pd

Configuration

In [2]:
run_dir = Path('prod-runs/8KNL-20T4-no-ps/')  # Use the proxy-free version

## Load in Each Type of Task
We'll need the runtimes and method sizes

In [3]:
tasks = []
for path in run_dir.glob('*-result*.json'):
    records = pd.read_json(path, lines=True)
    records['total_size'] = records['message_sizes'].apply(lambda x: sum(x.values())) / 1e6
    tasks.append(records[['method', 'time_running', 'total_size']])
tasks = pd.concat(tasks)
print(f'Loaded {len(tasks)} tasks')

Loaded 1680 tasks


## Group by Method
Get the median and mean for each type of task

In [4]:
tasks.groupby('method').describe(percentiles=[0.5])

time_running                                                 \
                      count        mean        std         min         50%   
method                                                                       
evaluate_mpnn        1104.0   39.934202   3.057144   30.567365   39.571590   
retrain_mpnn           56.0  353.943642  93.815752  137.087016  372.137334   
run_simulation        520.0   87.551868  47.670382   17.547251   78.009014   

                           total_size                                   \
                       max      count        mean       std        min   
method                                                                   
evaluate_mpnn    76.279101     1104.0  104.136820  2.657086  98.539011   
retrain_mpnn    499.819405       56.0    5.622344  0.018184   5.595638   
run_simulation  277.160734      520.0    1.485636  0.742893   0.309582   

                                        
                       50%         max  
method                                  
evaluate_mpnn   103.594807  108.989321  
retrain_mpnn      5.619790    5.654468  
run_simulation    1.372107    4.323642

## Get the total amount of compute time for inference
The time per task is not meaningful as we can choose chunk sizes

In [5]:
n_models = 8
n_chunks = 23 

In [6]:
inf_tasks = tasks.query("method=='evaluate_mpnn'")
n_runs = len(inf_tasks) / n_models / n_chunks
print(f'Pulled {n_runs} runs worth of data')

Pulled 6.0 runs worth of data


In [7]:
inf_tasks[['time_running', 'total_size']].sum() / n_models / n_runs

time_running     918.486636
total_size      2395.146870
dtype: float64

Print the molecules per second

In [8]:
1e6 / (inf_tasks['time_running'].sum() / n_models / n_runs)

1088.7474688686348